In [ ]:
import fitbit
import json

import pandas as pd
import cherrypy
import os
import sys
import threading
import traceback
import webbrowser
import time
from pydlc import dense_lines
from matplotlib import pyplot as plt
import plotly.express as px
from scipy.signal import savgol_filter
import numpy as np

from urllib.parse import urlparse
from base64 import b64encode
from fitbit.api import Fitbit
from oauthlib.oauth2.rfc6749.errors import MismatchingStateError, MissingTokenError
from datetime import datetime, date, timedelta
from ast import literal_eval
from scipy.interpolate import interp1d

In [ ]:
class OAuth2Server:
    def __init__(self, client_id, client_secret,
                 redirect_uri='http://127.0.0.1:8080/'):
        """ Initialize the FitbitOauth2Client """
        self.success_html = """
            <h1>You are now authorized to access the Fitbit API!</h1>
            <br/><h3>You can close this window</h3>"""
        self.failure_html = """
            <h1>ERROR: %s</h1><br/><h3>You can close this window</h3>%s"""

        self.fitbit = Fitbit(
            client_id,
            client_secret,
            redirect_uri=redirect_uri,
            timeout=10,
        )

        self.redirect_uri = redirect_uri

    def browser_authorize(self):
        """
        Open a browser to the authorization url and spool up a CherryPy
        server to accept the response
        """
        url, _ = self.fitbit.client.authorize_token_url()
        # Open the web browser in a new thread for command-line browser support
        threading.Timer(1, webbrowser.open, args=(url,)).start()

        # Same with redirect_uri hostname and port.
        urlparams = urlparse(self.redirect_uri)
        cherrypy.config.update({'server.socket_host': urlparams.hostname,
                                'server.socket_port': urlparams.port})

        cherrypy.quickstart(self)

    @cherrypy.expose
    def index(self, state, code=None, error=None):
        """
        Receive a Fitbit response containing a verification code. Use the code
        to fetch the access_token.
        """
        error = None
        if code:
            try:
                self.fitbit.client.fetch_access_token(code)
            except MissingTokenError:
                error = self._fmt_failure(
                    'Missing access token parameter.</br>Please check that '
                    'you are using the correct client_secret')
            except MismatchingStateError:
                error = self._fmt_failure('CSRF Warning! Mismatching state')
        else:
            error = self._fmt_failure('Unknown error while authenticating')
        # Use a thread to shutdown cherrypy so we can return HTML first
        self._shutdown_cherrypy()
        return error if error else self.success_html

    def _fmt_failure(self, message):
        tb = traceback.format_tb(sys.exc_info()[2])
        tb_html = '<pre>%s</pre>' % ('\n'.join(tb)) if tb else ''
        return self.failure_html % (message, tb_html)

    def _shutdown_cherrypy(self):
        """ Shutdown cherrypy in one second, if it's running """
        if cherrypy.engine.state == cherrypy.engine.states.STARTED:
            threading.Timer(1, cherrypy.engine.exit).start()

In [ ]:
with open('fitbit_tokens.json') as f:
    TOKENS = literal_eval(f.read())
CLIENT = TOKENS['CLIENT']
CLIENT_ID = CLIENT['CLIENT_ID']
CLIENT_SECRET = CLIENT['CLIENT_SECRET']
REDIRECT_URI = 'http://127.0.0.1:8080/'
server = OAuth2Server(CLIENT_ID,CLIENT_SECRET)
server.browser_authorize()
profile = server.fitbit.user_profile_get()
TOKENS = server.fitbit.client.session.token
TOKENS = server.fitbit.client.session.token
ACCESS_TOKEN = TOKENS['access_token']
REFRESH_TOKEN = TOKENS['refresh_token']
EXPIRES_AT = TOKENS['expires_at']
client = Fitbit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    access_token=ACCESS_TOKEN,
    refresh_token=REFRESH_TOKEN,
    expires_at=EXPIRES_AT,
    redirect_uri=REDIRECT_URI
)

In [ ]:
end_date = date.today()

In [ ]:
start_date = end_date - timedelta(days=365)

In [ ]:
date_range = [start_date + timedelta(days=x) for x in range((end_date - start_date).days +1)]

In [ ]:
date_range

In [ ]:
all_hr_df = pd.DataFrame()

for date in date_range:
    print(date.strftime("%Y-%m-%d"))
    day_hr = client.intraday_time_series('activities/heart',base_date=date)['activities-heart-intraday']['dataset']
    day_hr_df = pd.DataFrame(day_hr)
    day_hr_df['time'] = pd.to_datetime(day_hr_df['time'],format='%H:%M:%S').dt.time
    day_hr_df['time'] = day_hr_df.apply(lambda row: datetime.combine(date,row['time']),axis=1)
    all_hr_df = pd.concat([all_hr_df,day_hr_df],ignore_index=True)
    time.sleep(27)

In [ ]:
all_hr_df['date'] = all_hr_df['time'].dt.date
all_hr_df['daytime'] = all_hr_df['time'].dt.time
all_hr_df['time'] = all_hr_df['daytime']
all_hr_df.drop(columns='daytime',inplace=True)

In [ ]:
fig = px.line(all_hr_df,x='time',y='value',color='date')
fig.update_traces(connectgaps=False)
fig.show()

In [ ]:
pivoted_df = all_hr_df.pivot(index='date',columns='time',values='value')

In [ ]:
list_of_arrays = pivoted_df.values.tolist()
interp_arrays = []
x = np.array([*range(0,1440,1)])
for y in list_of_arrays:
    array = np.array(y)
    valid = ~np.isnan(array)
    x_valid = x[valid]
    y_valid = array[valid]
    interp_function = interp1d(x_valid,y_valid,kind='linear',fill_value='extrapolate')
    y_interpolated = interp_function(x)
    array[~valid] = y_interpolated[~valid]
    array = savgol_filter(array,30,2)
    interp_arrays.append(array)

In [ ]:
for i,array in enumerate(interp_arrays):
    if (pd.isnull(array).sum()) > 0:
        print(array)
        print('NULL')
    for num in array:
        if abs(num) > 200:
            print(f'array {i} has num:{num}')

In [ ]:
idx = 4
original = list_of_arrays[idx]
inter_smooth = interp_arrays[idx]

plt.plot(original, label='Original',color='blue')
plt.plot(inter_smooth, label='Interpolated & Smoothed', color='red')
plt.show()

In [ ]:
interp

In [ ]:
fig,ax = plt.subplots()
fig.set_figheight(10)
fig.set_figwidth(25)
im = dense_lines(ys=interp_arrays,ax=ax,cmap='twilight_shifted')